# Maskin Læring: Klassifisering av dyr

## Trening av vårt første CNN!
![l](https://www.managers.org.uk/wp-content/uploads/2020/03/BrainTraining.jpg)  
I denne notebooken kommer vi til å gjøre et par ting: 
1. Skrive inn studentnummeret vårt
2. Generere et nevralt nettverk(tenk en hjerne) med dette studentnummeret. 
3. Trene dette nettverket

Først så setter vi inn student nummeret vårt.

In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import tensorflow as tf
import random

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
os.environ["CUDA_VISIBLE_DEVICES"]="1"
kaggle_user= "uzaaft"
kaggle_api = "eef66e4e79f6db8c868a369edc57500c"
os.environ["KAGGLE_USERNAME"] = kaggle_user
os.environ["KAGGLE_KEY"] = kaggle_api

In [ ]:
!rm -rf translate.py
!unzip -q animals10.zip 

replace raw-img/cane/OIF-e2bexWrojgtQnAPPcUfOWQ.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
base_dir = './raw-img'
LABELS = os.listdir(base_dir)
LABELS.sort()
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant",
                 "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat",
                 "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel",
                 "cane": "dog", "cavallo": "horse", "elephant" : "elefante", 
                 "butterfly": "farfalla", "chicken": "gallina", "cat": "gatto",
                 "cow": "mucca", "spider": "ragno", "squirrel": "scoiattolo"}
LABELS_english = list(translate.values())
base_dir + '/' + LABELS[0]
dataset=[]
testset=[]
count=0

for label in tqdm(LABELS):
    i=0
#     path = base_dir + '/' + label
    path = os.path.join(base_dir, label)
#     print(path)
    for img in os.listdir(path):
#         print(img)
        image=load_img(os.path.join(path, img), grayscale=False, color_mode='rgb', target_size=(75,75))
        image=img_to_array(image)
        image=image/255.0
        if i<1000:
            dataset.append([image,count])
            i+=1
        else:
            testset.append([image,count])
    count=count+1

X,y =zip(*dataset)
test, test_labels=zip(*testset)
y = to_categorical(y)
y=np.array(y)
X=np.array(X)
print("Train Shape:{}\nTrain Labels shape: {}".format(X.shape,y.shape))
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=99)

datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    fill_mode="nearest"
)

In [ ]:
def predict_image(image):
    image=img_to_array(image)/255 
    prediction_image=np.array(image)
    prediction_image= np.expand_dims(image, axis=0)
    
    pred = model.predict(prediction_image)
    value = np.argmax(pred)
    return f"Pred: {LABELS_english[value]}"

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='elu', input_shape=(75, 75, 3), padding= "valid"))
model.add(tf.keras.layers.MaxPooling2D(3))

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='elu', padding= "valid"))
model.add(tf.keras.layers.MaxPooling2D(2))

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding= "valid"))
model.add(tf.keras.layers.MaxPooling2D(1))


model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding= "valid"))
model.add(tf.keras.layers.MaxPooling2D(1))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64, activation='elu'))
model.add(tf.keras.layers.Dense(32, activation='elu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))



model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
             optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-5),
             metrics=['acc'])

### I cellen ovenfor lager vi modellen. Vi prøver å gjette hvilket dyr det er på bildene med denne ulærte modellen

In [ ]:
image_dir = "raw-img"
sub_dirs = os.listdir(image_dir)
image_paths = []
complete_paths = []
for sub_dir in sub_dirs:
    image_path = os.listdir(f"{image_dir}/{sub_dir}")[0]
    complete_paths.append(f"{image_dir}/{sub_dir}/{image_path}")

plt.figure(figsize = (16,16))
for i in range(0,10):
    image=load_img(complete_paths[i],target_size=(75,75))
    prediction = predict_image(image)
    plt.subplot(5,5,i+1)    # the number of images in the grid is 5*5 (25)
    plt.title(prediction)
    plt.imshow(image)
plt.axis("off")
plt.show()

In [ ]:
history = model.fit(
     datagen.flow(X_train, y_train, batch_size=128), 
    validation_data = (X_valid, y_valid),
    epochs=25
)

### La oss se hvor bra modellen vår gjetter

In [ ]:
plt.plot(history.history['acc'], label = "Acc")
plt.plot(history.history['val_acc'], label = "Val Acc")

In [ ]:
plt.plot(history.history['loss'], label = "loss")
plt.plot(history.history['val_loss'], label = "Val loss")

In [ ]:
image_dir = "raw-img"
sub_dirs = os.listdir(image_dir)
image_paths = []
complete_paths = []
for sub_dir in sub_dirs:
    image_path = os.listdir(f"{image_dir}/{sub_dir}")[0]
    complete_paths.append(f"{image_dir}/{sub_dir}/{image_path}")

plt.figure(figsize = (16,16))
for i in range(0,10):
    image=load_img(complete_paths[i],target_size=(75,75))
    prediction = predict_image(image)
    plt.subplot(5,5,i+1)    # the number of images in the grid is 5*5 (25)
    plt.title(prediction)
    plt.imshow(image)
plt.axis("off")
plt.show()

In [ ]:
history = model.fit(
     datagen.flow(X_train, y_train, batch_size=128), 
    validation_data = (X_valid, y_valid),
    epochs=100, 
    initial_epoch = 25
)

In [ ]:
plt.plot(history.history['acc'], label = "Acc")
plt.plot(history.history['val_acc'], label = "Val Acc")

In [ ]:
plt.plot(history.history['loss'],  label = "Loss")
plt.plot(history.history['val_loss'],  label = "Val loss")

In [ ]:
image=load_img("raw-img/elefante/OIP---LeldVL441fx5S66TGgVQAAAA.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/gatto/1006.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()
image=load_img("raw-img/farfalla/OIP--7K5JzW1ZEmrY2YRyVjFCQHaE7.jpeg",target_size=(75,75))
predict_image(image)
plt.imshow(image)
plt.show()

In [ ]:
history = model.fit(
     datagen.flow(X_train, y_train, batch_size=128), 
    validation_data = (X_valid, y_valid),
    epochs=1000, 
    initial_epoch = 500
)

In [ ]:
plt.plot(history.history['acc'], label = "Acc")
plt.plot(history.history['val_acc'], label = "Val Acc")

In [ ]:
plt.plot(history.history['loss'],  label = "Loss")
plt.plot(history.history['val_loss'],  label = "Val loss")

In [ ]:
image_dir = "raw-img"
sub_dirs = os.listdir(image_dir)
image_paths = []
complete_paths = []
for sub_dir in sub_dirs:
    image_path = os.listdir(f"{image_dir}/{sub_dir}")[0]
    complete_paths.append(f"{image_dir}/{sub_dir}/{image_path}")

plt.figure(figsize = (16,16))
for i in range(0,10):
    image=load_img(complete_paths[i],target_size=(75,75))
    prediction = predict_image(image)
    plt.subplot(5,5,i+1)    # the number of images in the grid is 5*5 (25)
    plt.title(prediction)
    plt.imshow(image)
plt.axis("off")
plt.show()